## Load in modules

In [1]:
# basics
import math
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
import json

# plotting
import matplotlib.pyplot as plt

# shapefiles
import geopandas as gpd
import shapely

# mobility networks
import infomap
import networkx as nx
from ete3 import Tree


## Read in extracted TLs from MCC tree for all subtrees

In [2]:
def get_cog_id(name):
    return name.split('|')[0].split('/')[1]


In [3]:
BA1_mcc_tls = pd.read_csv('../../BA.1/mcc_tls/mcc_tls.tsv', sep='\t').iloc[:,2:]
BA1_mcc_tls = BA1_mcc_tls.loc[(BA1_mcc_tls.ntaxa > 0) & (BA1_mcc_tls.source == 'nonENG')]
BA1_mcc_tls.taxa = BA1_mcc_tls.taxa.apply(lambda x: x.split('; '))
BA1_mcc_tls['cog_ids'] = BA1_mcc_tls.taxa.apply(lambda x: [get_cog_id(xx) for xx in x])
BA1_mcc_tls['duration'] = BA1_mcc_tls.apply(lambda row: row.last_seen - row.tmrca, axis=1)
BA1_mcc_tls['import_est'] = BA1_mcc_tls.apply(lambda row: (row.tmrca + row.ptmrca)/2, axis=1)
BA1_mcc_tls = BA1_mcc_tls.sort_values('ntaxa', ascending=False)


## Examine size distribution of TLs for each subtree (excluding singletons)

In [4]:
BA1_tls_size_distr = {}
for ntaxa in BA1_mcc_tls[BA1_mcc_tls.ntaxa > 1].ntaxa.values:
    if ntaxa in BA1_tls_size_distr:
        BA1_tls_size_distr[ntaxa] += 1
    else:
        BA1_tls_size_distr[ntaxa] = 1

print(BA1_tls_size_distr)
BA1_mcc_tls.ntaxa.sum()


{9727: 1, 1406: 1, 944: 1, 256: 1, 212: 1, 180: 1, 138: 1, 108: 1, 106: 1, 104: 1, 103: 1, 94: 1, 73: 1, 60: 2, 45: 1, 43: 1, 42: 1, 39: 1, 38: 1, 37: 1, 30: 1, 28: 3, 27: 3, 25: 1, 21: 2, 20: 2, 19: 1, 18: 5, 16: 4, 14: 5, 13: 3, 12: 4, 11: 6, 10: 4, 9: 5, 8: 12, 7: 19, 6: 26, 5: 35, 4: 65, 3: 102, 2: 236}


17597

Singletons: 515 (BA.1.15) + 511 (BA.1.17) + 1882 (BA.1.1) + 1401 (BA.1) = 4309 (total)
TLs: 210 (BA.1.15) + 200 (BA.1.17) + 936 (BA.1.1) + 565 (BA.1) = 1911 (total)

Take 8 largest transmission lineages (each n > 500 sequences)
- BA.1.15_DTA_102_n2975 (57.5%)
- BA.1.15_DTA_700_n713 (13.8%)
- BA.1.17_DTA_175_n11351 (83.4%)
- BA.1.1_DTA_1254_n2249 (18.4%)
- BA.1.1_DTA_1467_n722 (5.9%)
- BA.1_DTA_1207_n9727 (55.3%)
- BA.1_DTA_1944_n1406 (8.0%)
- BA.1_DTA_1800_n944 (5.4%)


## Read in COG-UK geography metadata

In [5]:
BA1_cog_geo = pd.read_csv('../../BA.1/BA.1.geography.csv', sep=',')
BA1_cog_pc = dict(zip(BA1_cog_geo.cog_id.values, BA1_cog_geo.outer_postcode.values))


## Read in mobility data (to check LTLA consistency)

In [6]:
ltla_flow = pd.read_csv('../../../../Data/LTLA_England_112019_052022_renamed_14Sep2022.csv', sep=',')
ltla_flow = ltla_flow[(ltla_flow.week_of > '2021-10-01') & (ltla_flow.week_of < '2022-01-30')]


## Read in outer postcode <-> LTLA mapping

In [7]:
with open('../../area-search_fixed_NP25_added.json', 'r') as infile:
    pc_ltla = json.loads(infile.read().strip())
    
pc_ltla


{'AL1': 'E07000240',
 'AL10': 'E07000241',
 'AL2': 'E07000240',
 'AL3': 'E07000240',
 'AL4': 'E07000240',
 'AL5': 'E07000240',
 'AL6': 'E07000241',
 'AL7': 'E07000241',
 'AL8': 'E07000241',
 'AL9': 'E07000241',
 'B1': 'E08000025',
 'B10': 'E08000025',
 'B11': 'E08000025',
 'B12': 'E08000025',
 'B13': 'E08000025',
 'B14': 'E08000025',
 'B15': 'E08000025',
 'B16': 'E08000025',
 'B17': 'E08000025',
 'B18': 'E08000025',
 'B19': 'E08000025',
 'B2': 'E08000025',
 'B20': 'E08000025',
 'B21': 'E08000025',
 'B22': 'E08000025',
 'B23': 'E08000025',
 'B24': 'E08000025',
 'B25': 'E08000025',
 'B26': 'E08000025',
 'B27': 'E08000025',
 'B28': 'E08000025',
 'B29': 'E08000025',
 'B3': 'E08000025',
 'B30': 'E08000025',
 'B31': 'E08000025',
 'B32': 'E08000025',
 'B33': 'E08000025',
 'B34': 'E08000025',
 'B35': 'E08000025',
 'B36': 'E08000029',
 'B37': 'E08000029',
 'B38': 'E08000025',
 'B4': 'E08000025',
 'B40': 'E08000029',
 'B42': 'E08000025',
 'B43': 'E08000028',
 'B44': 'E08000025',
 'B45': 'E080000

## Examine LTLA distribution within each major lineage

### LTLAs to ignore

In [8]:
## ignore Isles of Scilly, Isles of Wright, City of London, and South Tyneside
ignore_ltlas = [
    'E06000053', 'E09000001', 'E08000023', 'E06000046'
]
NA_ltlas = [
    'E09000001', 'E08000023'
]


In [9]:
def get_ltla_distr(cog_ids, cog_pc_map, keep_omitted=False):
    ltla_distr = {}
    for cog_id in cog_ids:
        pc = cog_pc_map[cog_id]
        ltla = pc_ltla[pc] if pc in pc_ltla else 'NA'
        if keep_omitted or (not keep_omitted and ltla not in ignore_ltlas):
            if ltla in ltla_distr:
                ltla_distr[ltla] += 1
            else:
                ltla_distr[ltla] = 1

    return ltla_distr
            

In [10]:
len(BA1_cog_pc)


17645

In [11]:
BA1_mcc_tls['ltla_distr'] = BA1_mcc_tls.cog_ids.apply(lambda x: get_ltla_distr(x, BA1_cog_pc))


In [12]:
BA1_mcc_tls['ltla_distr_keep_omitted'] = BA1_mcc_tls.cog_ids.apply(lambda x: get_ltla_distr(x, BA1_cog_pc,
                                                                                            keep_omitted=True))


### Get lineage info

In [13]:
mcc_tl = BA1_mcc_tls[BA1_mcc_tls.ntaxa == 9727].copy()
mcc_tl


,ntaxa,tmrca,ptmrca,source,first_seen,last_seen,taxa,cog_ids,duration,import_est,ltla_distr,ltla_distr_keep_omitted
1207,9727,2021.870022,2021.864643,nonENG,2021.920548,2022.082192,"[England/PHEC-3V076VD7/2021|BA.1|2021-12-06, E...","[PHEC-3V076VD7, MILK-2E0C273, ALDP-2D9F1E2, AL...",0.21217,2021.867333,"{'E09000031': 67, 'E09000019': 51, 'E06000015'...","{'E09000031': 67, 'E09000019': 51, 'E06000015'..."


In [14]:
len(mcc_tl.ltla_distr_keep_omitted.values[0])

313

In [15]:
count = 0
for cog_id in mcc_tl.cog_ids.values[0]:
    pc = BA1_cog_pc[cog_id]
    if pc not in pc_ltla:
        print(pc)
        count += 1
    else:
        if pc_ltla[pc] == 'E06000053':
            print(pc, 'Isles of Scilly')
            count += 1
        elif pc_ltla[pc] == 'E06000046':
            print(pc, 'Isles of Wright')
            count += 1
        elif pc_ltla[pc] == 'E09000001':
            print(pc, 'City of London')
            count += 1
        elif pc_ltla[pc] == 'E08000023':
            print(pc, 'South Tyneside')
            count += 1

count


EC1A City of London
EC1A City of London
EC3R City of London
PO30 Isles of Wright
NE34 South Tyneside
NE34 South Tyneside
PO31 Isles of Wright
EC3N City of London
EC3N City of London
PO36 Isles of Wright
NE32 South Tyneside
PO30 Isles of Wright
NE32 South Tyneside
NE33 South Tyneside
EC4M City of London
NE31 South Tyneside
NE31 South Tyneside
NE35 South Tyneside
PO36 Isles of Wright
PO30 Isles of Wright
PO31 Isles of Wright
NE32 South Tyneside
NE36 South Tyneside
EC4V City of London
NE32 South Tyneside
EC1A City of London
PO30 Isles of Wright
NE31 South Tyneside
NE31 South Tyneside


29

#### Omitted LTLAs and sequences
- BA.1.15_DTA_102_n2975 (8 sequences found in omitted LTLAs)
- BA.1.15_DTA_700_n713 (2 sequences found in omitted LTLAs)
- BA.1.17_DTA_175_n11351 (39 sequences found in omitted LTLAs)
- BA.1.1_DTA_1254_n2249 (18 sequences found in omitted LTLAs)
- BA.1.1_DTA_1467_n722 (0 sequences found in omitted LTLAs)
- BA.1_DTA_1207_n9727 (29 sequences found in omitted LTLAs; 1 sequence removed due to mis-labelled country, it should be Wales rather than England)
- BA.1_DTA_1944_n1406 (6 sequences found in omitted LTLAs)
- BA.1_DTA_1800_n944 (13 sequences found in omitted LTLAs)


## Read in LTLA neighbour list

In [16]:
ltla_nbs = pd.read_csv('../../ltla_neighbours_v4_ACTIVE.tsv', sep='\t')
ltla_nbs = dict(zip(ltla_nbs.ltla_code.values, [x.split(',') for x in ltla_nbs.neighbours.values]))


## Aggregate neighbouring LTLAs starting from LTLAs with fewest sequence samples
1. look for LTLA with fewer sequence samples
2. look for neighbouring LTLAs with the most sequence samples and merge
3. update new merged LTLAs
4. go back to step 1 and iterate

Operate on dictionaries, keep track of mergers

In [17]:
def aggr_ltlas(ltla_distr, nlim=250):
    mergers = {}
    ltla_distr_cpy = { k: v for k, v in ltla_distr.items() }
    seqs_moved = 0
    while len(ltla_distr_cpy) > nlim:
        if len(ltla_distr) <= nlim:
            return ltla_distr
        else:
            ## find LTLA with fewest genome sample(s), ignoring omitted LTLAs
            min_ltla_num = sorted(list(filter(lambda x: x[0] not in ignore_ltlas,
                                              [(k, v) for k, v in ltla_distr_cpy.items()])),
                                  key=lambda x: x[1])
            for ltla_num in min_ltla_num:
                ## find adjacent LTLA with most genome sample(s)
                ## no omitted LTLAs in neighbour list, so don't worry
                nbs = ltla_nbs[ltla_num[0]]
                max_nb = sorted([(nb, ltla_distr_cpy[nb]
                                  if nb in ltla_distr_cpy else -1) for nb in nbs], key=lambda x: x[1])
                max_nb = list(filter(lambda x: x[1] > 0, max_nb))
                if len(max_nb):
                    max_nb = max_nb[-1]
                    ltla_distr_cpy[max_nb[0]] += ltla_num[1]
                    del ltla_distr_cpy[ltla_num[0]]
                    
                    ## check if LTLA to be merged into is already in another merger(s)
                    ## if yes, then we want to add new LTLA into the same merger
                    if max_nb[0] in mergers:
                        mergers[max_nb[0]].append(ltla_num[0])
                    else:
                        mergers[max_nb[0]] = [ltla_num[0]]

                    ## check if LTLA has nested mergers
                    ## if yes, then we want to add those nested mergers into the same merger
                    if ltla_num[0] in mergers:
                        mergers[max_nb[0]] += mergers[ltla_num[0]]
                        del mergers[ltla_num[0]]
                                        
                    seqs_moved += ltla_distr[ltla_num[0]] ## add original number of sequence (excluding collapsed)
                    break
            else: ## if no merger was found
                return {'results': 'initial LTLA(s): %d, final LTLA(s): %d (failure)' % \
                        (len(ltla_distr), len(ltla_distr_cpy)), 'seqs_moved': seqs_moved,
                        'mergers': mergers, 'ltla_distr': ltla_distr_cpy}
            
    ## finalise mergers (potential nested mergers) 
        
    return {'results': 'initial LTLA(s): %d, final LTLA(s): %d (success)' % \
            (len(ltla_distr), len(ltla_distr_cpy)), 'seqs_moved': seqs_moved,
            'mergers': mergers, 'ltla_distr': ltla_distr_cpy}
    

In [18]:
mergers = aggr_ltlas(mcc_tl.ltla_distr_keep_omitted.values[0], nlim=253)
mergers


{'results': 'initial LTLA(s): 313, final LTLA(s): 253 (success)',
 'seqs_moved': 590,
 'mergers': {'E06000047': ['E07000166', 'E07000030', 'E06000001', 'E06000005'],
  'E06000052': ['E07000046', 'E07000047'],
  'E06000043': ['E07000223'],
  'E06000042': ['E07000156'],
  'E06000014': ['E07000164'],
  'E07000026': ['E07000029'],
  'E07000123': ['E07000119', 'E07000124'],
  'E07000130': ['E07000133'],
  'E06000025': ['E07000080', 'E07000082'],
  'E06000051': ['E07000235'],
  'E07000078': ['E07000083'],
  'E07000141': ['E06000017'],
  'E07000227': ['E07000225'],
  'E07000246': ['E07000043', 'E07000042', 'E07000040'],
  'E06000011': ['E07000167', 'E07000168'],
  'E06000004': ['E06000002', 'E06000003'],
  'E07000245': ['E07000009'],
  'E08000035': ['E07000165', 'E07000169'],
  'E07000064': ['E07000062', 'E07000116'],
  'E06000026': ['E07000044'],
  'E06000016': ['E07000135', 'E07000131'],
  'E08000008': ['E07000037'],
  'E06000012': ['E07000142', 'E07000137'],
  'E08000019': ['E07000035'],
 

In [19]:
for k, v in mergers['ltla_distr'].items():
    if k in ignore_ltlas:
        print(k, v)


E09000001 8
E06000046 8
E08000023 13


In [20]:
## remap mergers so that keys correspond to LTLAs that have been merged,
## and values correspond to LTLAs that have been merged into
mergers_rev = {}
for k, v in mergers['mergers'].items():
    for vv in v:
        mergers_rev[vv] = k
    

In [22]:
with open('mergers_rev.tsv', 'w+') as outfile:
    outfile.write('child_ltla\tparent_ltla\n')
    outfile.write('\n'.join(['%s\t%s' % (k, v) for k, v in mergers_rev.items()]))
    

In [21]:
## note however that there are sub-lineages of BA.1.1    
for taxa in mcc_tl.taxa.values[0]:
    if 'BA.1' not in taxa:
        print(taxa)
    

## Prepare tip trait (LTLA and date)

In [22]:
## epochs definitions (left/right inclusive)
epochs = {
    1: (dt.datetime(2021, 11, 1), dt.datetime(2021, 12, 25)),
    2: (dt.datetime(2021, 12, 26), dt.datetime(2022, 1, 15)),
    3: (dt.datetime(2022, 1, 16), dt.datetime(2022, 1, 31))
}

def bin_date(date_str, epochs):
    date = dt.datetime.strptime(date_str, '%Y-%m-%d')
    for k, v in epochs.items():
        if (date >= v[0] and date <= v[1]):
            return k


In [23]:
tip_ltla = dict(zip(mcc_tl.taxa.values[0],
                    [(mergers_rev[pc_ltla[BA1_cog_pc[cog_id]]] if pc_ltla[BA1_cog_pc[cog_id]] in mergers_rev
                      else pc_ltla[BA1_cog_pc[cog_id]])
                     for cog_id in mcc_tl.cog_ids.values[0]]))


In [24]:
tip_epoch = dict(zip(mcc_tl.taxa.values[0], [bin_date(x.split('|')[-1], epochs) for x in mcc_tl.taxa.values[0]]))
                     

In [25]:
with open('tip_ltla_date.tsv', 'w+') as outfile:
    outfile.write('name\tltla\tdate\tepoch\n')
    outfile.write('\n'.join(['%s\t%s\t%s\t%d' % (k, v, k.split('|')[-1], tip_epoch[k]) for k, v in tip_ltla.items()]))
    

## Prepare and export LTLA-specific traits
population, cumulative case number, community membership, network centrality, local mobility per capita

### Read in LTLA population

In [27]:
ltla_pops = pd.read_csv('../../../../lalago/age/ONS_mid2020_10032022_v2_059_60plus.csv', sep=',')
ltla_pops_merged = ltla_pops[['code', 'total']].copy()
ltla_pops_merged.code = ltla_pops_merged.code.apply(lambda x: mergers_rev[x] if x in mergers_rev else x)
ltla_pops_merged = ltla_pops_merged.groupby('code').agg({'total': 'sum'}).reset_index()
ltla_pops_merged


,code,total
0,E06000004,475932
1,E06000006,129759
2,E06000007,209397
3,E06000008,413681
4,E06000009,138381
...,...,...
249,E09000029,207707
250,E09000030,331969
251,E09000031,276940
252,E09000032,329735


In [28]:
ltla_pops_merged_dict = dict(zip(ltla_pops_merged.code.values, ltla_pops_merged.total.values))


In [29]:
with open('ltla_pop.tsv', 'w+') as outfile:
    outfile.write('ltla\tpop\n')
    outfile.write('\n'.join(['%s\t%d' % (k, ltla_pops_merged_dict[k]) for k in mergers['ltla_distr'].keys()]))  


### Calculate population-weighted peak timing (days from 2021-12-01)

In [30]:
ltla_pops_dict = dict(zip(ltla_pops.code.values, ltla_pops.total.values))

ltla_peak_times_all = pd.read_csv('../ltla_peak_mid_dec_ref_all_ltlas_thres_0.85.csv', sep=',')
ltla_peak_times_all_dict = dict(zip(ltla_peak_times_all.origin.values, ltla_peak_times_all.dec_dist.values))
ltla_peak_times_all_dict = { k: v+1 for k, v in ltla_peak_times_all_dict.items() }


In [31]:
## get population-weighted average for aggregated LTLAs
mergers_pop_w_peak_times = {}
for k, v in mergers['mergers'].items():
    ltlas_list = [k] + v
    ltlas_pops = [ltla_pops_dict[ltla] for ltla in ltlas_list]
    ltlas_times = [ltla_peak_times_all_dict[ltla] for ltla in ltlas_list]
    pop_w_time = sum(np.array(ltlas_pops)*np.array(ltlas_times))/sum(ltlas_pops)
    mergers_pop_w_peak_times[k] = pop_w_time
    
ltla_peak_times_all = ltla_peak_times_all[
    ltla_peak_times_all.origin.isin(list(mergers['ltla_distr'].keys()))].copy()
ltla_peak_times_all['pop_w_dec_dist'] = ltla_peak_times_all.apply(
    lambda row: mergers_pop_w_peak_times[row.origin] if row.origin in mergers_pop_w_peak_times
    else ltla_peak_times_all_dict[row.origin],
    axis=1)
    

In [32]:
## write to file
with open('ltla_peak_times.tsv', 'w+') as outfile:
    outfile.write('ltla\tpeak_t_from_dec\n')
    outfile.write(
        '\n'.join(['%s\t%f' % (row.origin, row.pop_w_dec_dist) for index, row in ltla_peak_times_all.iterrows()]))


### Aggregate mobility data according to mergers

In [33]:
ltla_flow_merged = ltla_flow.copy()
ltla_flow_merged.origin = ltla_flow_merged.origin.apply(lambda x: mergers_rev[x] if x in mergers_rev else x)
ltla_flow_merged.destination = ltla_flow_merged.destination.apply(
    lambda x: mergers_rev[x] if x in mergers_rev else x)


### Prepare local mobility

In [34]:
start_date = '2021-11-28'
end_date = '2022-01-31'
mob_scale_fac = 3657300764
local_flow_study = ltla_flow_merged[(ltla_flow_merged.week_of >= start_date) & \
                                    (ltla_flow_merged.week_of <= end_date) & \
                                    (ltla_flow_merged.origin.isin(list(mergers['ltla_distr'].keys()))) & \
                                    (ltla_flow_merged.destination.isin(list(mergers['ltla_distr'].keys()))) & \
                                    (ltla_flow_merged.origin == ltla_flow_merged.destination)
                                   ].copy()
local_flow_study.movement = local_flow_study.movement.apply(lambda x: x*mob_scale_fac)

## compute average over study period
local_flow_study_agg = local_flow_study.groupby(['origin', 'destination']).agg({'movement': 'sum'}).reset_index()
local_flow_study_agg.movement = local_flow_study_agg.movement.apply(
    lambda x: x/len(local_flow_study.week_of.unique()))

local_flow_study_agg_dict = dict(zip(local_flow_study_agg.origin.values, local_flow_study_agg.movement.values))

local_flow_study_agg


,origin,destination,movement
0,E06000004,E06000004,2.417536e+06
1,E06000006,E06000006,4.127678e+05
2,E06000007,E06000007,1.050150e+06
3,E06000008,E06000008,1.759445e+06
4,E06000009,E06000009,4.561008e+05
...,...,...,...
246,E09000029,E09000029,7.716319e+05
247,E09000030,E09000030,7.106591e+05
248,E09000031,E09000031,8.073650e+05
249,E09000032,E09000032,9.920778e+05


### Prepare mobility matrices

In [35]:
def year_fraction(date):
    start = dt.date(date.year, 1, 1).toordinal()
    year_length = dt.date(date.year+1, 1, 1).toordinal() - start
    return date.year + float(date.toordinal() - start) / year_length


In [36]:
## find earlist week that includes estimated importaion date
import_est = mcc_tl.import_est.values[0]
import_week_of = ''
for index, week_of in enumerate(ltla_flow_merged.week_of.unique()):
    week_of_dt = dt.datetime.strptime(week_of, '%Y-%m-%d')
    week_of_dec = year_fraction(week_of_dt)
    next_week_of_dt = dt.datetime.strptime(ltla_flow_merged.week_of.unique()[index+1], '%Y-%m-%d')
    next_week_of_dec = year_fraction(next_week_of_dt)
    
    if (import_est >= week_of_dec) and (import_est < next_week_of_dec):
        import_week_of = week_of
        break
    
print(import_week_of)


2021-11-07


#### Epoch-1

In [38]:
start_date = import_week_of
end_date = '2021-12-25'
mob_scale_fac = 3657300764
ltla_flow_epo_1 = ltla_flow_merged[(ltla_flow_merged.week_of >= start_date) & \
                                   (ltla_flow_merged.week_of <= end_date) & \
                                   (ltla_flow_merged.origin.isin(list(mergers['ltla_distr'].keys()))) & \
                                   (ltla_flow_merged.destination.isin(list(mergers['ltla_distr'].keys())))
                                  ].copy()
ltla_flow_epo_1.movement = ltla_flow_epo_1.movement.apply(lambda x: x*mob_scale_fac)

## remove self-loops
ltla_flow_epo_1 = ltla_flow_epo_1[ltla_flow_epo_1.origin != ltla_flow_epo_1.destination]

## compute average over study period
ltla_flow_epo_1_agg = ltla_flow_epo_1.groupby(['origin', 'destination']).agg({'movement': 'sum'}).reset_index()
ltla_flow_epo_1_agg.movement = ltla_flow_epo_1_agg.movement.apply(lambda x: x/len(ltla_flow_epo_1.week_of.unique()))

## sum movement for (origin, destination) (destination, origin) pairs
for index, row in ltla_flow_epo_1_agg.iterrows():
    sorted_ltlas = sorted([row.origin, row.destination])
    if row.origin != sorted_ltlas[0]: ## need swapping
        ltla_flow_epo_1_agg.at[index, 'destination'] = sorted_ltlas[1]
        ltla_flow_epo_1_agg.at[index, 'origin'] = sorted_ltlas[0]
ltla_flow_epo_1_agg = ltla_flow_epo_1_agg.groupby(['origin', 'destination']).agg({'movement': 'sum'}).reset_index()
ltla_flow_epo_1_agg.movement = ltla_flow_epo_1_agg.movement.apply(lambda x: x/2)

ltla_flow_epo_1_agg


,origin,destination,movement
0,E06000004,E06000011,2856.090661
1,E06000004,E06000014,11531.208073
2,E06000004,E06000047,66757.755788
3,E06000004,E08000021,641.856284
4,E06000004,E08000024,782.401128
...,...,...,...
1980,E09000030,E09000032,11604.092853
1981,E09000030,E09000033,101176.613278
1982,E09000031,E09000032,3290.264509
1983,E09000031,E09000033,35034.328961


In [39]:
G_w_epo_1 = nx.DiGraph()
G_w_epo_1.add_nodes_from(list(mergers['ltla_distr'].keys())) ## add from complete lineage-LTLA list as some LTLAs
## might not have connection to any other LTLAs, but we still want to include it as it represents 0 mobility
epo_1_weighted_edges = [(ltla_flow_epo_1_agg['origin'].values[i],
                         ltla_flow_epo_1_agg['destination'].values[i],
                         ltla_flow_epo_1_agg['movement'].values[i]) for i in range(len(ltla_flow_epo_1_agg))]
G_w_epo_1.add_weighted_edges_from(epo_1_weighted_edges)
G_undir_w_epo_1 = G_w_epo_1.to_undirected()


In [40]:
## get adjacency matrix
ajM_epo_1 = np.array(nx.adjacency_matrix(G_undir_w_epo_1).todense())

for ltla in NA_ltlas:
    if ltla in list(G_undir_w_epo_1.nodes):
        index = list(G_undir_w_epo_1.nodes).index(ltla)
        ajM_epo_1[index] = np.array([np.nan]*len(ajM_epo_1))
        for i in range(len(ajM_epo_1)):
            ajM_epo_1[i][index] = np.nan
    

In [41]:
## set pseudo count in off-diagonal elements
for i in range(len(ajM_epo_1)):
    for j in range(len(ajM_epo_1)):
        if ajM_epo_1[i][j] == 0.0 and i != j:
            ajM_epo_1[i][j] = 0.001


In [42]:
## write to file
with open('mobility_matrix_epo_1.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_1.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_1.nodes())[i] + ',' + ','.join(
            [('NA' if np.isnan(x) else str(x)) for x in t]))for i, t in enumerate(ajM_epo_1)]))


#### Epoch-2

In [43]:
start_date = '2021-12-26'
end_date = '2022-01-15'
mob_scale_fac = 3657300764
ltla_flow_epo_2 = ltla_flow_merged[(ltla_flow_merged.week_of >= start_date) & \
                                   (ltla_flow_merged.week_of <= end_date) & \
                                   (ltla_flow_merged.origin.isin(list(mergers['ltla_distr'].keys()))) & \
                                   (ltla_flow_merged.destination.isin(list(mergers['ltla_distr'].keys())))
                                  ].copy()
ltla_flow_epo_2.movement = ltla_flow_epo_2.movement.apply(lambda x: x*mob_scale_fac)

## remove self-loops
ltla_flow_epo_2 = ltla_flow_epo_2[ltla_flow_epo_2.origin != ltla_flow_epo_2.destination]

## compute average over study period
ltla_flow_epo_2_agg = ltla_flow_epo_2.groupby(['origin', 'destination']).agg({'movement': 'sum'}).reset_index()
ltla_flow_epo_2_agg.movement = ltla_flow_epo_2_agg.movement.apply(lambda x: x/len(ltla_flow_epo_2.week_of.unique()))

## sum movement for (origin, destination) (destination, origin) pairs
for index, row in ltla_flow_epo_2_agg.iterrows():
    sorted_ltlas = sorted([row.origin, row.destination])
    if row.origin != sorted_ltlas[0]: ## need swapping
        ltla_flow_epo_2_agg.at[index, 'destination'] = sorted_ltlas[1]
        ltla_flow_epo_2_agg.at[index, 'origin'] = sorted_ltlas[0]
ltla_flow_epo_2_agg = ltla_flow_epo_2_agg.groupby(['origin', 'destination']).agg({'movement': 'sum'}).reset_index()
ltla_flow_epo_2_agg.movement = ltla_flow_epo_2_agg.movement.apply(lambda x: x/2)

ltla_flow_epo_2_agg


,origin,destination,movement
0,E06000004,E06000011,2747.851974
1,E06000004,E06000014,5003.187445
2,E06000004,E06000047,39076.430013
3,E06000004,E08000021,133.491478
4,E06000004,E08000024,424.856439
...,...,...,...
1610,E09000030,E09000032,5809.012713
1611,E09000030,E09000033,68025.794210
1612,E09000031,E09000032,2129.768145
1613,E09000031,E09000033,23010.517307


In [44]:
G_w_epo_2 = nx.DiGraph()
G_w_epo_2.add_nodes_from(list(mergers['ltla_distr'].keys())) ## add from complete lineage-LTLA list as some LTLAs
## might not have connection to any other LTLAs, but we still want to include it as it represents 0 mobility
epo_2_weighted_edges = [(ltla_flow_epo_2_agg['origin'].values[i],
                         ltla_flow_epo_2_agg['destination'].values[i],
                         ltla_flow_epo_2_agg['movement'].values[i]) for i in range(len(ltla_flow_epo_2_agg))]
G_w_epo_2.add_weighted_edges_from(epo_2_weighted_edges)
G_undir_w_epo_2 = G_w_epo_2.to_undirected()


In [45]:
## get adjacency matrix
ajM_epo_2 = np.array(nx.adjacency_matrix(G_undir_w_epo_2).todense())

for ltla in NA_ltlas:
    if ltla in list(G_undir_w_epo_2.nodes):
        index = list(G_undir_w_epo_2.nodes).index(ltla)
        ajM_epo_2[index] = np.array([np.nan]*len(ajM_epo_2))
        for i in range(len(ajM_epo_2)):
            ajM_epo_2[i][index] = np.nan
    

In [46]:
## set pseudo count in off-diagonal elements
for i in range(len(ajM_epo_2)):
    for j in range(len(ajM_epo_2)):
        if ajM_epo_2[i][j] == 0.0 and i != j:
            ajM_epo_2[i][j] = 0.001


In [47]:
## write to file
with open('mobility_matrix_epo_2.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_2.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_2.nodes())[i] + ',' + ','.join(
            [('NA' if np.isnan(x) else str(x)) for x in t]))for i, t in enumerate(ajM_epo_2)]))


#### Epoch-3

In [48]:
start_date = '2022-01-16'
end_date = '2022-01-31'
mob_scale_fac = 3657300764
ltla_flow_epo_3 = ltla_flow_merged[(ltla_flow_merged.week_of >= start_date) & \
                                   (ltla_flow_merged.week_of <= end_date) & \
                                   (ltla_flow_merged.origin.isin(list(mergers['ltla_distr'].keys()))) & \
                                   (ltla_flow_merged.destination.isin(list(mergers['ltla_distr'].keys())))
                                  ].copy()
ltla_flow_epo_3.movement = ltla_flow_epo_3.movement.apply(lambda x: x*mob_scale_fac)

## remove self-loops
ltla_flow_epo_3 = ltla_flow_epo_3[ltla_flow_epo_3.origin != ltla_flow_epo_3.destination]

## compute average over study period
ltla_flow_epo_3_agg = ltla_flow_epo_3.groupby(['origin', 'destination']).agg({'movement': 'sum'}).reset_index()
ltla_flow_epo_3_agg.movement = ltla_flow_epo_3_agg.movement.apply(lambda x: x/len(ltla_flow_epo_3.week_of.unique()))

## sum movement for (origin, destination) (destination, origin) pairs
for index, row in ltla_flow_epo_3_agg.iterrows():
    sorted_ltlas = sorted([row.origin, row.destination])
    if row.origin != sorted_ltlas[0]: ## need swapping
        ltla_flow_epo_3_agg.at[index, 'destination'] = sorted_ltlas[1]
        ltla_flow_epo_3_agg.at[index, 'origin'] = sorted_ltlas[0]
ltla_flow_epo_3_agg = ltla_flow_epo_3_agg.groupby(['origin', 'destination']).agg({'movement': 'sum'}).reset_index()
ltla_flow_epo_3_agg.movement = ltla_flow_epo_3_agg.movement.apply(lambda x: x/2)

ltla_flow_epo_3_agg


,origin,destination,movement
0,E06000004,E06000011,2480.564243
1,E06000004,E06000014,7873.254220
2,E06000004,E06000047,50365.603146
3,E06000004,E08000021,407.789035
4,E06000004,E08000024,645.513585
...,...,...,...
1606,E09000030,E09000032,10551.312704
1607,E09000030,E09000033,82654.997266
1608,E09000031,E09000032,3235.796851
1609,E09000031,E09000033,29139.543837


In [49]:
G_w_epo_3 = nx.DiGraph()
G_w_epo_3.add_nodes_from(list(mergers['ltla_distr'].keys())) ## add from complete lineage-LTLA list as some LTLAs
## might not have connection to any other LTLAs, but we still want to include it as it represents 0 mobility
epo_3_weighted_edges = [(ltla_flow_epo_3_agg['origin'].values[i],
                         ltla_flow_epo_3_agg['destination'].values[i],
                         ltla_flow_epo_3_agg['movement'].values[i]) for i in range(len(ltla_flow_epo_3_agg))]
G_w_epo_3.add_weighted_edges_from(epo_3_weighted_edges)
G_undir_w_epo_3 = G_w_epo_3.to_undirected()


In [50]:
## get adjacency matrix
ajM_epo_3 = np.array(nx.adjacency_matrix(G_undir_w_epo_3).todense())

for ltla in NA_ltlas:
    if ltla in list(G_undir_w_epo_3.nodes):
        index = list(G_undir_w_epo_3.nodes).index(ltla)
        ajM_epo_3[index] = np.array([np.nan]*len(ajM_epo_3))
        for i in range(len(ajM_epo_3)):
            ajM_epo_3[i][index] = np.nan
    

In [51]:
## set pseudo count in off-diagonal elements
for i in range(len(ajM_epo_3)):
    for j in range(len(ajM_epo_3)):
        if ajM_epo_3[i][j] == 0.0 and i != j:
            ajM_epo_3[i][j] = 0.001


In [52]:
## write to file
with open('mobility_matrix_epo_3.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_3.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_3.nodes())[i] + ',' + ','.join(
            [('NA' if np.isnan(x) else str(x)) for x in t]))for i, t in enumerate(ajM_epo_3)]))


### Prepare infomap output

#### Epoch-1

In [53]:
im_multiLevel_directed_weighted_epo_1 = infomap.Infomap()
mapping_epo_1 = im_multiLevel_directed_weighted_epo_1.add_networkx_graph(G_undir_w_epo_1)
mapping_rev_epo_1 = { v: k for k, v in mapping_epo_1.items() }
im_multiLevel_directed_weighted_epo_1.run()

lvl_1_mods_epo_1 = im_multiLevel_directed_weighted_epo_1.get_modules(depth_level=1)
lvl_2_mods_epo_1 = im_multiLevel_directed_weighted_epo_1.get_modules(depth_level=2)


In [54]:
im_multiLevel_directed_weighted_epo_1.write_newick('infomap_multiLevel_epo_1_avg.tre')
input_tree_epo_1 = Tree('infomap_multiLevel_epo_1_avg.tre', format=1)
output_tree_epo_1 = 'infomap_multiLevel_epo_1_avg.renamed.tre'
for leaf in input_tree_epo_1.get_leaves():
    leaf.name = mapping_epo_1[int(leaf.name)]
    
input_tree_epo_1.write(outfile=output_tree_epo_1, format=1)


In [55]:
## create membership-overlap matrix
multiLevel_lvl_1_mod_overlap_matrix_epo_1 = []
for node in G_undir_w_epo_1.nodes:
    row = [(1 if lvl_1_mods_epo_1[mapping_rev_epo_1[n]] == lvl_1_mods_epo_1[mapping_rev_epo_1[node]] else 0) \
           for n in G_undir_w_epo_1]
    multiLevel_lvl_1_mod_overlap_matrix_epo_1.append(row)
    
multiLevel_lvl_1_mod_overlap_matrix_epo_1 = np.array(multiLevel_lvl_1_mod_overlap_matrix_epo_1) 

## write to file
with open('multiLevel_lvl_1_infomap_overlap_epo_1.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_1.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_1.nodes())[i] + ',' + ','.join([str(x) for x in t])) \
        for i, t in enumerate(multiLevel_lvl_1_mod_overlap_matrix_epo_1)]))
    
## write to file
with open('multiLevel_lvl_1_infomap_overlap_NA_epo_1.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_1.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_1.nodes())[i] + ',' + ','.join(
            [('NA' if (list(G_undir_w_epo_1.nodes())[i] in NA_ltlas or 
                       list(G_undir_w_epo_1.nodes())[ii] in NA_ltlas) else str(x))
             for ii, x in enumerate(t)])) \
        for i, t in enumerate(multiLevel_lvl_1_mod_overlap_matrix_epo_1)]))


In [56]:
## create membership-overlap matrix
multiLevel_lvl_2_mod_overlap_matrix_epo_1 = []
for node in G_undir_w_epo_1.nodes:
    row = [(1 if lvl_2_mods_epo_1[mapping_rev_epo_1[n]] == lvl_2_mods_epo_1[mapping_rev_epo_1[node]] else 0) \
           for n in G_undir_w_epo_1]
    multiLevel_lvl_2_mod_overlap_matrix_epo_1.append(row)
    
multiLevel_lvl_2_mod_overlap_matrix_epo_1 = np.array(multiLevel_lvl_2_mod_overlap_matrix_epo_1) 

## write to file
with open('multiLevel_lvl_2_infomap_overlap_epo_1.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_1.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_1.nodes())[i] + ',' + ','.join([str(x) for x in t])) \
        for i, t in enumerate(multiLevel_lvl_2_mod_overlap_matrix_epo_1)]))
    
## write to file
with open('multiLevel_lvl_2_infomap_overlap_NA_epo_1.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_1.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_1.nodes())[i] + ',' + ','.join(
            [('NA' if (list(G_undir_w_epo_1.nodes())[i] in NA_ltlas or 
                       list(G_undir_w_epo_1.nodes())[ii] in NA_ltlas) else str(x))
             for ii, x in enumerate(t)])) \
        for i, t in enumerate(multiLevel_lvl_2_mod_overlap_matrix_epo_1)]))


#### Epoch-2

In [57]:
im_multiLevel_directed_weighted_epo_2 = infomap.Infomap()
mapping_epo_2 = im_multiLevel_directed_weighted_epo_2.add_networkx_graph(G_undir_w_epo_2)
mapping_rev_epo_2 = { v: k for k, v in mapping_epo_2.items() }
im_multiLevel_directed_weighted_epo_2.run()

lvl_1_mods_epo_2 = im_multiLevel_directed_weighted_epo_2.get_modules(depth_level=1)
lvl_2_mods_epo_2 = im_multiLevel_directed_weighted_epo_2.get_modules(depth_level=2)


In [58]:
im_multiLevel_directed_weighted_epo_2.write_newick('infomap_multiLevel_epo_2_avg.tre')
input_tree_epo_2 = Tree('infomap_multiLevel_epo_2_avg.tre', format=1)
output_tree_epo_2 = 'infomap_multiLevel_epo_2_avg.renamed.tre'
for leaf in input_tree_epo_2.get_leaves():
    leaf.name = mapping_epo_2[int(leaf.name)]
    
input_tree_epo_2.write(outfile=output_tree_epo_2, format=1)


In [59]:
## create membership-overlap matrix
multiLevel_lvl_1_mod_overlap_matrix_epo_2 = []
for node in G_undir_w_epo_2.nodes:
    row = [(1 if lvl_1_mods_epo_2[mapping_rev_epo_2[n]] == lvl_1_mods_epo_2[mapping_rev_epo_2[node]] else 0) \
           for n in G_undir_w_epo_2]
    multiLevel_lvl_1_mod_overlap_matrix_epo_2.append(row)
    
multiLevel_lvl_1_mod_overlap_matrix_epo_2 = np.array(multiLevel_lvl_1_mod_overlap_matrix_epo_2) 

## write to file
with open('multiLevel_lvl_1_infomap_overlap_epo_2.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_2.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_2.nodes())[i] + ',' + ','.join([str(x) for x in t])) \
        for i, t in enumerate(multiLevel_lvl_1_mod_overlap_matrix_epo_2)]))
    
## write to file
with open('multiLevel_lvl_1_infomap_overlap_NA_epo_2.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_2.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_2.nodes())[i] + ',' + ','.join(
            [('NA' if (list(G_undir_w_epo_2.nodes())[i] in NA_ltlas or 
                       list(G_undir_w_epo_2.nodes())[ii] in NA_ltlas) else str(x))
             for ii, x in enumerate(t)])) \
        for i, t in enumerate(multiLevel_lvl_1_mod_overlap_matrix_epo_2)]))


In [60]:
## create membership-overlap matrix
multiLevel_lvl_2_mod_overlap_matrix_epo_2 = []
for node in G_undir_w_epo_2.nodes:
    row = [(1 if lvl_2_mods_epo_2[mapping_rev_epo_2[n]] == lvl_2_mods_epo_2[mapping_rev_epo_2[node]] else 0) \
           for n in G_undir_w_epo_2]
    multiLevel_lvl_2_mod_overlap_matrix_epo_2.append(row)
    
multiLevel_lvl_2_mod_overlap_matrix_epo_2 = np.array(multiLevel_lvl_2_mod_overlap_matrix_epo_2) 

## write to file
with open('multiLevel_lvl_2_infomap_overlap_epo_2.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_2.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_2.nodes())[i] + ',' + ','.join([str(x) for x in t])) \
        for i, t in enumerate(multiLevel_lvl_2_mod_overlap_matrix_epo_2)]))
    
## write to file
with open('multiLevel_lvl_2_infomap_overlap_NA_epo_2.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_2.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_2.nodes())[i] + ',' + ','.join(
            [('NA' if (list(G_undir_w_epo_2.nodes())[i] in NA_ltlas or 
                       list(G_undir_w_epo_2.nodes())[ii] in NA_ltlas) else str(x))
             for ii, x in enumerate(t)])) \
        for i, t in enumerate(multiLevel_lvl_2_mod_overlap_matrix_epo_2)]))


#### Epoch-3

In [61]:
im_multiLevel_directed_weighted_epo_3 = infomap.Infomap()
mapping_epo_3 = im_multiLevel_directed_weighted_epo_3.add_networkx_graph(G_undir_w_epo_3)
mapping_rev_epo_3 = { v: k for k, v in mapping_epo_3.items() }
im_multiLevel_directed_weighted_epo_3.run()

lvl_1_mods_epo_3 = im_multiLevel_directed_weighted_epo_3.get_modules(depth_level=1)
lvl_2_mods_epo_3 = im_multiLevel_directed_weighted_epo_3.get_modules(depth_level=2)


In [62]:
im_multiLevel_directed_weighted_epo_3.write_newick('infomap_multiLevel_epo_3_avg.tre')
input_tree_epo_3 = Tree('infomap_multiLevel_epo_3_avg.tre', format=1)
output_tree_epo_3 = 'infomap_multiLevel_epo_3_avg.renamed.tre'
for leaf in input_tree_epo_3.get_leaves():
    leaf.name = mapping_epo_3[int(leaf.name)]
    
input_tree_epo_3.write(outfile=output_tree_epo_3, format=1)


In [63]:
## create membership-overlap matrix
multiLevel_lvl_1_mod_overlap_matrix_epo_3 = []
for node in G_undir_w_epo_3.nodes:
    row = [(1 if lvl_1_mods_epo_3[mapping_rev_epo_3[n]] == lvl_1_mods_epo_3[mapping_rev_epo_3[node]] else 0) \
           for n in G_undir_w_epo_3]
    multiLevel_lvl_1_mod_overlap_matrix_epo_3.append(row)
    
multiLevel_lvl_1_mod_overlap_matrix_epo_3 = np.array(multiLevel_lvl_1_mod_overlap_matrix_epo_3) 

## write to file
with open('multiLevel_lvl_1_infomap_overlap_epo_3.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_3.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_3.nodes())[i] + ',' + ','.join([str(x) for x in t])) \
        for i, t in enumerate(multiLevel_lvl_1_mod_overlap_matrix_epo_3)]))
    
## write to file
with open('multiLevel_lvl_1_infomap_overlap_NA_epo_3.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_3.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_3.nodes())[i] + ',' + ','.join(
            [('NA' if (list(G_undir_w_epo_3.nodes())[i] in NA_ltlas or 
                       list(G_undir_w_epo_3.nodes())[ii] in NA_ltlas) else str(x))
             for ii, x in enumerate(t)])) \
        for i, t in enumerate(multiLevel_lvl_1_mod_overlap_matrix_epo_3)]))


In [64]:
## create membership-overlap matrix
multiLevel_lvl_2_mod_overlap_matrix_epo_3 = []
for node in G_undir_w_epo_3.nodes:
    row = [(1 if lvl_2_mods_epo_3[mapping_rev_epo_3[n]] == lvl_2_mods_epo_3[mapping_rev_epo_3[node]] else 0) \
           for n in G_undir_w_epo_3]
    multiLevel_lvl_2_mod_overlap_matrix_epo_3.append(row)
    
multiLevel_lvl_2_mod_overlap_matrix_epo_3 = np.array(multiLevel_lvl_2_mod_overlap_matrix_epo_3) 

## write to file
with open('multiLevel_lvl_2_infomap_overlap_epo_3.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_3.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_3.nodes())[i] + ',' + ','.join([str(x) for x in t])) \
        for i, t in enumerate(multiLevel_lvl_2_mod_overlap_matrix_epo_3)]))
    
## write to file
with open('multiLevel_lvl_2_infomap_overlap_NA_epo_3.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_3.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_3.nodes())[i] + ',' + ','.join(
            [('NA' if (list(G_undir_w_epo_3.nodes())[i] in NA_ltlas or 
                       list(G_undir_w_epo_3.nodes())[ii] in NA_ltlas) else str(x))
             for ii, x in enumerate(t)])) \
        for i, t in enumerate(multiLevel_lvl_2_mod_overlap_matrix_epo_3)]))


### Prepare mobility network centrality

#### Epoch-1

In [65]:
eigenvector_centrality_epo_1 = nx.eigenvector_centrality(G_undir_w_epo_1, weight='weight')
betweenness_centrality_epo_1 = nx.betweenness_centrality(G_undir_w_epo_1, weight='weight')


In [66]:
with open('ltla_eigenCentral_epo_1.tsv', 'w+') as outfile:
    outfile.write('ltla\teigenCentral\n')
    outfile.write('\n'.join(
        ['%s\t%s' % (k, 'NA' if k in NA_ltlas else str(v))
         for k, v in eigenvector_centrality_epo_1.items()]))
    

In [67]:
with open('ltla_betweenCentral_epo_1.tsv', 'w+') as outfile:
    outfile.write('ltla\tbetweenCentral\n')
    outfile.write('\n'.join(
        ['%s\t%s' % (k, 'NA' if k in NA_ltlas else str(v))
         for k, v in betweenness_centrality_epo_1.items()]))
    

#### Epoch-2

In [68]:
eigenvector_centrality_epo_2 = nx.eigenvector_centrality(G_undir_w_epo_2, weight='weight')
betweenness_centrality_epo_2 = nx.betweenness_centrality(G_undir_w_epo_2, weight='weight')


In [69]:
with open('ltla_eigenCentral_epo_2.tsv', 'w+') as outfile:
    outfile.write('ltla\teigenCentral\n')
    outfile.write('\n'.join(
        ['%s\t%s' % (k, 'NA' if k in NA_ltlas else str(v))
         for k, v in eigenvector_centrality_epo_2.items()]))
    

In [70]:
with open('ltla_betweenCentral_epo_2.tsv', 'w+') as outfile:
    outfile.write('ltla\tbetweenCentral\n')
    outfile.write('\n'.join(
        ['%s\t%s' % (k, 'NA' if k in NA_ltlas else str(v))
         for k, v in betweenness_centrality_epo_2.items()]))
    

#### Epoch-3

In [71]:
eigenvector_centrality_epo_3 = nx.eigenvector_centrality(G_undir_w_epo_3, weight='weight')
betweenness_centrality_epo_3 = nx.betweenness_centrality(G_undir_w_epo_3, weight='weight')


In [72]:
with open('ltla_eigenCentral_epo_3.tsv', 'w+') as outfile:
    outfile.write('ltla\teigenCentral\n')
    outfile.write('\n'.join(
        ['%s\t%s' % (k, 'NA' if k in NA_ltlas else str(v))
         for k, v in eigenvector_centrality_epo_3.items()]))
    

In [73]:
with open('ltla_betweenCentral_epo_3.tsv', 'w+') as outfile:
    outfile.write('ltla\tbetweenCentral\n')
    outfile.write('\n'.join(
        ['%s\t%s' % (k, 'NA' if k in NA_ltlas else str(v))
         for k, v in betweenness_centrality_epo_3.items()]))
    

### Prepare great-circle distance matrix

In [74]:
def get_displacement(pos_1, pos_2, km=True):
    R = 6371e3 ## Earth's radius in meters
    phi_1 = pos_1.y*math.pi/180
    phi_2 = pos_2.y*math.pi/180
    delta_phi = (pos_2.y - pos_1.y)*math.pi/180
    delta_lambda = (pos_2.x - pos_1.x)*math.pi/180
    
    a = math.sin(delta_phi/2)**2 + math.cos(phi_1)*math.cos(phi_2)*math.sin(delta_lambda/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R*c
    
    return (d/1000 if km else d) ## return shortest distance in metres


In [75]:
gdf = gpd.read_file('../../../../Data/shp/shapefile_out.shp')
gdf = gdf.rename(columns = {'geo_code':'origin'})
gdf = gdf.to_crs(epsg=27700)
gdf['centroid'] = gdf['geometry'].centroid.to_crs(epsg=4326)


In [76]:
gdf_merged = gdf.copy()
for p, m in mergers['mergers'].items():
    p_index = gdf_merged[gdf_merged.origin == p].index.values[0]
    gdf_merger = gdf_merged[gdf_merged.origin.isin([p] + m)]
    gdf_merged.loc[p_index, 'centroid'] = gdf_merger.dissolve().centroid.to_crs(epsg=4326).values[0]
    gdf_merged = gdf_merged[~gdf_merged.origin.isin(m)]


In [77]:
## get displacements using centroid coordinates
origin_centroid_dict = dict(zip(gdf_merged['origin'].values, gdf_merged['centroid'].values))
gdf_merged = gdf_merged.to_crs(epsg=4326)


In [78]:
ltla_lat_long = dict(zip(gdf_merged[gdf_merged.origin.isin(mergers['ltla_distr'])].origin,
                         gdf_merged[gdf_merged.origin.isin(mergers['ltla_distr'])].centroid))
with open('ltla_lat_long.tsv', 'w+') as outfile:
    outfile.write('ltla\tLAT\tLONG\n')
    outfile.write('\n'.join(['%s\t%f\t%f' % (k, v.x, v.y) for k, v in ltla_lat_long.items()]))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [79]:
# get_displacement(origin_centroid_dict[row.origin], origin_centroid_dict[row.destination], km=True)
ajM_distance = []
for node in G_undir_w_epo_1.nodes():
    tmp = []
    for n in G_undir_w_epo_1.nodes():
        if node == n:
            tmp.append(0.0)
        else:
            dist = get_displacement(origin_centroid_dict[node],
                                    origin_centroid_dict[n], km=True)
            tmp.append(dist)
    ajM_distance.append(tmp)
    
ajM_distance = np.array(ajM_distance)


In [80]:
## write to file
with open('grtCircle_distance.csv', 'w+') as outfile:
    outfile.write(',' + ','.join(list(G_undir_w_epo_1.nodes())) + '\n')
    outfile.write('\n'.join([
        (list(G_undir_w_epo_1.nodes())[i] + ',' + ','.join([str(x) for x in t])) \
        for i, t in enumerate(ajM_distance)]))


### Map LTLA to region


In [26]:
ltla_region = pd.read_csv('../../../../Data/ons/la_code_region_buckinghamshire_fixed.csv', sep=',')
ltla_region_dict = dict(zip(ltla_region.la_code, ltla_region.region_name))


In [27]:
region_name_tl = {
    'North East': 'NE',
    'North West': 'NW',
    'Yorkshire and The Humber': 'YH',
    'East Midlands': 'EM',
    'West Midlands': 'WM',
    'South West': 'SW',
    'East': 'E',
    'South East': 'SE',
    'London': 'L',
    'Wales': 'W'
}

ltla_region_dict = { k: region_name_tl[v] for k, v in ltla_region_dict.items() }


In [28]:
regions = ['NE', 'NW', 'YH', 'EM', 'WM', 'SW', 'E', 'SE', 'L']
count = 0
with open('ltla_region.tsv', 'w+') as outfile:
    outfile.write('ltla\t%s\n' % ('\t'.join(regions)))
    for index, ltla in enumerate(mergers['ltla_distr'].keys()):
        ltla_region = ltla_region_dict[ltla]
        tmp_v = [ 1 if region == ltla_region else 0 for region in regions ]
        count += sum(tmp_v)
        outfile.write('%s\t%s' % (ltla, '\t'.join([str(x) for x in tmp_v])))
        if index < len(mergers['ltla_distr'].keys()) - 1:
            outfile.write('\n')
    
print(count)
    

253
